# Query data from overpassAPI (openstreetmaps)


query now with Python by using the requests package in order to access the Overpass AP

## Area[] Berlin 
area["ISO3166-2"="DE-BE"]
https://www.openstreetmap.org/relation/62422

## admin_level=4
admin_level=2 through admin_level=10 allow for different administrative subdivision schemes to be handled in a consistent way by data consumers. The use of a numbering scheme rather than words for the values helps avoid confusion due to different terminology used in different countries
A data consumer looking for municipalities corresponding to "city", "town" or "village" boundaries will find these tagged anywhere from admin_level=4 (e.g. relation Berlin)

In [ ]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-2"="DE-BE"][admin_level=4];
(node['shop'='convenience'](area);
 way['shop'='convenience' ](area);
 rel['shop'='convenience' ](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
overpass_query

In [ ]:
data

## Plotting 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Collect coords into list
coords = []
for element in data['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords.append((lon, lat))# Convert coordinates into numpy array
X = np.array(coords)

plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Spatkauf in Berlin')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

In [ ]:
kkk

## params function

In [ ]:
#not working
# def param_key(keys):
    
#     #params defining the whole key example 'leisure' 
# #     get every type of leisure instalation, as a list returns several keys
    
#     string_keys = ''
#     for k in keys: 
#         string_keys += f"""node[{k}](area);"""
#     return string_keys

# def param_key_value(key, value):
    
#     #params defining the key and value, example 'leisure' 'parks' will only get parks
    
#     string_key = f"""\
#     node['{key}'='{value}'](area);
#     way['{key}'='{value}'](area);
#     rel['{key}'='{value}'](area);
#     """
#     return string_key

In [1]:
import requests
import json

In [2]:
#style working properly
# osm_keys_ok = """node["amenity"="pub"](area.berlin);
#             node["amenity"="bar"](area.berlin); 
#             """

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

study_area = 'area["ISO3166-2"="DE-BE"]->.berlin'

keys_values_osm = {'amenity':['bbq','cafe']}


def param_nodes(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""node['{k}'='{v}'](area.berlin);"""
    return osm_keys
    
def param_ways(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""way['{k}'='{v}'](area.berlin);"""
    return osm_keys

def param_area(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""area['{k}'='{v}'](area.berlin);"""
    return osm_keys


def query_params_osm(keys, feat='nodes'):
    '''Adding keys and values as a dictionary, example: keys_values_osm = {'amenity':['bbq','cafe']}
    several values can added to a same key as a list, returns a dict
    arg feat = nodes, ways or areas'''
    
    if feat == 'ways':
        params = param_ways(keys)
        out_type = 'skel'
    if feat == 'areas':
        params = params_areas(keys)
        out_type = 'skel'
    else:
        params = param_nodes(keys)
        out_type = 'center'
        
    overpass_query = f"""
                    [out:json];
                    {study_area};
                    ({params}
                    );
                    out {out_type};
                    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    return response.json()

In [ ]:
keys_values_osm

In [ ]:
param_nodes(keys_values_osm)

In [12]:
data = query_params_osm(keys_values_osm, feat='nodes')

In [13]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.57 93a4d346',
 'osm3s': {'timestamp_osm_base': '2022-05-31T10:22:24Z',
  'timestamp_areas_base': '2022-05-31T09:05:24Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 30012754,
   'lat': 52.5388551,
   'lon': 13.3960965,
   'tags': {'addr:city': 'Berlin',
    'addr:country': 'DE',
    'addr:housenumber': '55',
    'addr:postcode': '13355',
    'addr:street': 'Brunnenstraße',
    'addr:suburb': 'Gesundbrunnen',
    'amenity': 'cafe',
    'contact:phone': '+49 30 46456678',
    'internet_access': 'wlan',
    'name': 'Eiscafe Eisberg',
    'website': 'http://www.eis-berg.de/',
    'wheelchair': 'limited'}},
  {'type': 'node',
   'id': 35528080,
   'lat': 52.5476798,
   'lon': 13.3563426,
   'tags': {'addr:city': 'Berlin',
    'addr:housenumber': '146',
    'addr:postcode': '13353',
    'addr:street': 'Müllerstraße',
    'air

In [14]:
len(data['elements'])

2274

# Params for Mobility

## Get dict for public transport

In [17]:
public_transport = {'amenity':['bus_station'],
                    'bus_bay':'',
                   'highway':['bus_stop','platform'],
                   'public_transport':['stop_position','platform','station','stop_area'],
                   'railway':['station','tram_stop', 'subway_entrance']}

In [36]:
param_nodes(public_transport)

"node['amenity'='bus_station'](area.berlin);node['highway'='bus_stop'](area.berlin);node['highway'='platform'](area.berlin);node['public_transport'='stop_position'](area.berlin);node['public_transport'='platform'](area.berlin);node['public_transport'='station'](area.berlin);node['public_transport'='stop_area'](area.berlin);node['railway'='station'](area.berlin);node['railway'='tram_stop'](area.berlin);node['railway'='subway_entrance'](area.berlin);"

In [19]:
data = query_params_osm(public_transport)

In [21]:
len(data['elements'])

15095

## Get dict for Cyclelanes

In [ ]:
cycle_paths = {'cycleway':['track']}

In [ ]:
param_ways(cycle_paths)

In [ ]:
data_cycle = query_params_osm(cycle_paths, feat='ways')

In [55]:
data_cycle

{'version': 0.6,
 'generator': 'Overpass API 0.7.57 93a4d346',
 'osm3s': {'timestamp_osm_base': '2022-05-31T12:29:54Z',
  'timestamp_areas_base': '2022-05-31T11:06:54Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': []}

# Params for Social Life

In [ ]:
eating = {'amenity':['cafe','restaurant', 'food_court', 'ice_cream']}
night_life = { 'amenity':['bar','pub','bbq','biergarten', 'nightclub']}